# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import numpy as pd
import pandas as pd
import pickle
import matplotlib.pyplot as py
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin
import re
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline, FeatureUnion
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
%matplotlib inline

%config IPCompletor.greedy = True

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

In [ ]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('disaster_details', engine)
X = df['message']
Y = df.drop(columns = ['id', 'message' , 'original' ,'genre'])
Y.head()

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    
    reg_exp_non_anum = r"[^a-zA-Z0-9]"
    
    #Convert all text to lower case
    text = text.lower()
    
    #Remove all non-alphanumeric characterics with a space to prevent meshed words
    text = re.sub(reg_exp_non_anum, " ", text)
    
    #Tokenize text
    tokens = word_tokenize(text)
    
    #Instantiate Stemmer and Lemmitizer 
    stemmer = PorterStemmer()
    lemmitizer = WordNetLemmatizer()
    
    #Convert raw tokens into lemmitized and stemmed forms w/ no white spaces
    new_tokens = []
    
    for token in tokens:
        lem_token_noun = lemmitizer.lemmatize(token)
        lem_token_verb = lemmitizer.lemmatize(lem_token_noun, pos = 'v')
        stem_lem_token = stemmer.stem(lem_token_verb)
        new_token = stem_lem_token.strip()
        new_tokens.append(new_token)
        
    return new_tokens

In [ ]:
#Test if tokenize function works
text = X.iloc[1]
tokenize(text)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer= tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state = 30)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 30)

#train clf
pipeline.fit(X_train, Y_train)

#predict response
Y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
#Obtain names of the categories
category_names = Y.columns.tolist()

Y_pred_df = pd.DataFrame(Y_pred, columns = category_names)
Y_pred_df.head()

In [ ]:
#Loop through columns and outputing results

for i in range(len(Y.columns)):
    print('Category: {}'.format(category_names[i].upper()), "\n\n",
         classification_report(Y_test.iloc[:,i], Y_pred_df.iloc[:,i]))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline.get_params()

In [ ]:
parameters = {
 'clf__estimator__n_estimators': (5 ,10)
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=1)

In [ ]:
cv.fit(X_train, Y_train)

In [ ]:
Y_pred2 = cv.predict(X_test)

In [ ]:
Y_pred2_df = pd.DataFrame(Y_pred2, columns = category_names)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
for i in range(len(Y.columns)):
    print('Category:{}'.format(category_names[i].upper()), "\n\n",
         classification_report(Y_test.iloc[:,i], Y_pred2_df.iloc[:,i]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [18]:
pipeline3 = Pipeline([
    ('hashing', HashingVectorizer(tokenizer= tokenize)),
    ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state = 30)))
])


In [20]:
pipeline3.get_params()

{'memory': None,
 'steps': [('hashing',
   HashingVectorizer(alternate_sign=True, analyzer='word', binary=False,
            decode_error='strict', dtype=<class 'numpy.float64'>,
            encoding='utf-8', input='content', lowercase=True,
            n_features=1048576, ngram_range=(1, 1), non_negative=False,
            norm='l2', preprocessor=None, stop_words=None, strip_accents=None,
            token_pattern='(?u)\\b\\w\\w+\\b',
            tokenizer=<function tokenize at 0x7efe154cf620>)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=30),
              n_jobs=1))],
 'hashing': HashingVectorizer(alternate_sign=True, analyzer='word', binary=False,
          decode_error='strict', dtype=<class 'numpy.float64'>,
          encoding='utf-8', input='content', lowercase=True,
          n_features=1048576, ngram_range=(1, 1), non_negative=False,
          norm='l2',

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 30)

#train clf
pipeline3.fit(X_train, Y_train)

#predict response
Y_pred3 = pipeline3.predict(X_test)

In [ ]:
Y_pred3_df = pd.DataFrame(Y_pred, columns = category_names)
Y_pred3_df.head()

In [ ]:
#Loop through columns and outputing results

for i in range(len(Y.columns)):
    print('Category: {}'.format(category_names[i].upper()), "\n\n",
         classification_report(Y_test.iloc[:,i], Y_pred3_df.iloc[:,i]))

### 9. Export your model as a pickle file

In [ ]:
filename = 'model.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.